# Generates a report with all machine names, and the users connected to each machine.

Event class contains the date when the event happened, the name of the machine where it happened, the user involver, and the event type (login/logout). 

In [157]:
class Event: 
    def __init__(self, event_date, event_type, machine_name, user):
        self.date = event_date
        self.type = event_type
        self.machine = machine_name
        self.user = user

Sample events to check the code. 

In [170]:
events = [
    Event('2020-01-21 12:45:56', 'login', 'myworkstation.local', 'jordan'),
    Event('2020-01-22 15:53:42', 'logout', 'webserver.local', 'jordan'),
    Event('2020-01-21 18:53:21', 'login', 'webserver.local', 'lane'),
    Event('2020-01-22 10:25:34', 'logout', 'myworkstation.local', 'jordan'),
    Event('2020-01-21 08:20:01', 'login', 'webserver.local', 'jordan'),
    Event('2020-01-23 11:24:35', 'logout', 'mailserver.local', 'chris'),
    Event('2020-01-20 18:53:21', 'login', 'webserver.local', 'ronaldo'),
    Event('2020-01-24 19:33:01', 'logout', 'myworkstation.local', 'marcelo'),
    Event('2020-01-24 20:43:01', 'logout', 'myworkstation.local', 'marcelo'),
    Event('2020-01-24 20:33:01', 'login', 'myworkstation.local', 'marcelo'),
    Event('2020-01-24 20:33:01', 'login', 'myworkstation.local', 'jonatas'),    
]

Use logging library to create the report.

In [171]:
from time import strftime, localtime
import logging

def create_logger(destination_folder, logfile_name):
    """Create a logfile object using logging library.
    Set logging levels to debug and console handler created."""
    
    time = strftime("%d %b %Y %H:%M:%S", localtime()) # Get current time
    start_time = time.replace(":", "_").replace(" ", "_") # Use _ instead of :
    logger_name = f"{start_time}_{logfile_name}" # Create logger name
    logfile_directory = f"{destination_folder}/{logger_name}.log" # Create logfile directory
    
    # Create logger
    logger = logging.getLogger(logfile_directory)
    logger.setLevel(logging.DEBUG) # Set debug level to DEBUG
    
    # Create a file handler and set level to DEBUG
    FileHandler = logging.FileHandler(logfile_directory)
    FileHandler.setLevel(logging.DEBUG)
    
    # Create a console handler and set level to DEBUG
    ConsoleHandler = logging.StreamHandler()
    ConsoleHandler.setLevel(logging.DEBUG)
    
    # Create formatter
    formatter = logging.Formatter("%(asctime)-15s - %(levelname)-8s  %(message)s",  
                                  datefmt="%H:%M:%S %a %d %b %Y")
    
    # Add formatter to console and file handler
    FileHandler.setFormatter(formatter)
    ConsoleHandler.setFormatter(formatter)
    
    # Add console and file handler to logger
    logger.addHandler(FileHandler)
    logger.addHandler(ConsoleHandler)
    
    # Return logger object
    return logger

Define current users functions based on Event class inputs.

In [172]:
### Input is a list of events, event class 
### Event class attributes: Date, User, Machine, and Type (login/logout) 
### machine: <machine name>
### user: <user name>
### date: <date>
### type: <login/logout>
import os

class CurrentUsers:
    """Create a report listing all current users and specific machines
    on a local system."""
    
    def __init__(self, events):
        self.events = events
        
    def get_event_date(self, event):
        """Returns the event date."""
        return event.date

    def current_users(self):
        """Returns the machine dictionary given an events list formed by
        an event objects withe the following attributes: <machine name>, 
        <user name>, <event type>, and <event type (login/logout)>."""

        # Sort the events based on event date
        self.events.sort(key=get_event_date) # Use get_event_date function 
        machines = {} # Create a empty dictonaries to store the machines

        # Iterate through our list of events 
        for event in self.events:
            # Check if this machine already exist
            if event.machine not in machines:
                machines[event.machine] = set() # Use a set to create and remove elements

            # Check for login/logout type 
            if event.type == "login":
                machines[event.machine].add(event.user)
            elif event.type == "logout" and event.user in machines[event.machine]:
                machines[event.machine].remove(event.user)

        # Returns the machine dictionaries 
        return machines 

    def generate_report(self, report_directory=os.getcwd(), report_name="Current_Users_Report"):
        """Generate a report listing all current users."""
        
        # Get current users 
        machines = self.current_users()
        
        # Create report logifle 
        logger = create_logger(report_directory, report_name)

        # Iterate over keys and values of the machine dictionary
        for machine, users in  machines.items():
            # Check if there's at least one user using the machine
            if len(users) > 0:
                user_list = ", ".join(users) # Create a string with all users 
                logger.info(f"{machine}: {user_list}")
        
        logging.shutdown()

Let's feed these events into our CurrentUser class and create a new instance. Now, we can create the report using the generate_report method.

In [175]:
report = CurrentUsers(events)
report.generate_report()

13:28:47 Sun 02 Aug 2020 - INFO      webserver.local: ronaldo, lane
13:28:47 Sun 02 Aug 2020 - INFO      myworkstation.local: jonatas
